#Fetch Rewards Coding Exercise - Data Analyst



# First: Review Existing Unstructured Data and Diagram a New Structured Relational Data Model


In this part, I'll first review unstructured JSON data(```receipts.json, brands.json, and users.json```)using [JSON formatter](https://jsoneditoronline.org/) to have better grasp of data hierarchy within each file. After this, I'll develop new structured dataframes with following codes, and later created data model diagram using MySQL Workbench.

## Import Library

In [1]:
import numpy as np
import pandas as pd
import json
!pip install pandasql
import pandasql as ps

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pandasql: filename=pandasql-0.7.3-py3-none-any.whl size=26784 sha256=3e601274a42bdefbe84150c499d1f433551c5f2dc77d1d7499c2c99c97f02994
  Stored in directory: /root/.cache/pip/wheels/5c/4b/ec/41f4e116c8053c3654e2c2a47c62b4fca34cc67ef7b55deb7f
Successfully built pandasql


## Parse JSON using Python

### Receipts

Examining ```receipts.JSON```in JSON formatter, we can see following issues that need to be tackled: 

1. There's a nested list in ```rewardsReceiptItemList```column with multiple-to-one relationship to receipts data

2. The value of ```_id``` is hidden under another dictionary with key ```$oid```; Values of date columns are also hidden under dictionaries with key ```$date```

3. The date format of several date columns need to be converted for readability.









In [2]:
receipt = []

#read in 'receipts.json' file and extract values hidden under dictionaries, including '_id' and other date columns
with open('receipts.json') as file:
 for line in file:
  row = json.loads(line)
  receipt.append({
   '_id': row['_id']['$oid'],
   'bonusPointsEarned': row.get('bonusPointsEarned', None),
   'bonusPointsEarnedReason': row.get('bonusPointsEarnedReason', None),
   'createDate': row['createDate']['$date'],
   'dateScanned': row['dateScanned']['$date'],
   'finishedDate':row.get('finishedDate', {}).get('$date', None),
   'modifyDate': row['modifyDate']['$date'],
   'pointsAwardedDate': row.get('pointsAwardedDate', {}).get('$date', None),
   'pointsEarned':row.get('pointsEarned', None),
   'purchaseDate':row.get('purchaseDate', {}).get('$date', None),
   "purchasedItemCount": row.get('purchasedItemCount',None),
   'rewardsReceiptStatus': row['rewardsReceiptStatus'],
   'totalSpent': row.get('totalSpent', None),
   'userId': row['userId']
  })


#transform to dataframe
df_receipt = pd.DataFrame(receipt)
df_receipt

,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",1609687531000,1609687531000,1.609688e+12,1609687536000,1.609688e+12,500.0,1.609632e+12,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",1609687483000,1609687483000,1.609687e+12,1609687488000,1.609687e+12,150.0,1.609601e+12,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,1609687537000,1609687537000,NaN,1609687542000,NaN,5,1.609632e+12,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,1609687534000,1609687534000,1.609688e+12,1609687539000,1.609688e+12,5.0,1.609632e+12,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,1609687506000,1609687506000,1.609688e+12,1609687511000,1.609688e+12,5.0,1.609601e+12,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,1614594147000,1614594147000,NaN,1614594148000,NaN,25.0,1.597622e+12,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,None,1614613361873,1614613361873,NaN,1614613361873,NaN,None,NaN,NaN,SUBMITTED,None,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,None,1614607657664,1614607657664,NaN,1614607657664,NaN,None,NaN,NaN,SUBMITTED,None,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,1614604048000,1614604048000,NaN,1614604049000,NaN,25.0,1.597622e+12,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [3]:
#Convert date format
def DateFormat(data, col):
  ans= [ ]
  for i in data[col].values:
    ans.append(pd.Timestamp(i//1000, unit='s'))
  return ans


date_col = ['createDate', 'dateScanned', 'finishedDate',
             'modifyDate', 'pointsAwardedDate', 'purchaseDate']
for i in date_col:
  df_receipt[i] = DateFormat(df_receipt,i)

df_receipt


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


,_id,bonusPointsEarned,bonusPointsEarnedReason,createDate,dateScanned,finishedDate,modifyDate,pointsAwardedDate,pointsEarned,purchaseDate,purchasedItemCount,rewardsReceiptStatus,totalSpent,userId
0,5ff1e1eb0a720f0523000575,500.0,"Receipt number 2 completed, bonus point schedu...",2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:31,2021-01-03 15:25:36,2021-01-03 15:25:31,500.0,2021-01-03 00:00:00,5.0,FINISHED,26.00,5ff1e1eacfcf6c399c274ae6
1,5ff1e1bb0a720f052300056b,150.0,"Receipt number 5 completed, bonus point schedu...",2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:43,2021-01-03 15:24:48,2021-01-03 15:24:43,150.0,2021-01-02 15:24:43,2.0,FINISHED,11.00,5ff1e194b6a9d73a3a9f1052
2,5ff1e1f10a720f052300057a,5.0,All-receipts receipt bonus,2021-01-03 15:25:37,2021-01-03 15:25:37,NaT,2021-01-03 15:25:42,NaT,5,2021-01-03 00:00:00,1.0,REJECTED,10.00,5ff1e1f1cfcf6c399c274b0b
3,5ff1e1ee0a7214ada100056f,5.0,All-receipts receipt bonus,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:34,2021-01-03 15:25:39,2021-01-03 15:25:34,5.0,2021-01-03 00:00:00,4.0,FINISHED,28.00,5ff1e1eacfcf6c399c274ae6
4,5ff1e1d20a7214ada1000561,5.0,All-receipts receipt bonus,2021-01-03 15:25:06,2021-01-03 15:25:06,2021-01-03 15:25:11,2021-01-03 15:25:11,2021-01-03 15:25:06,5.0,2021-01-02 15:25:06,2.0,FINISHED,1.00,5ff1e194b6a9d73a3a9f1052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1114,603cc0630a720fde100003e6,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 10:22:27,2021-03-01 10:22:27,NaT,2021-03-01 10:22:28,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33
1115,603d0b710a720fde1000042a,NaN,None,2021-03-01 15:42:41,2021-03-01 15:42:41,NaT,2021-03-01 15:42:41,NaT,None,NaT,NaN,SUBMITTED,None,5fc961c3b8cfca11a077dd33
1116,603cf5290a720fde10000413,NaN,None,2021-03-01 14:07:37,2021-03-01 14:07:37,NaT,2021-03-01 14:07:37,NaT,None,NaT,NaN,SUBMITTED,None,5fc961c3b8cfca11a077dd33
1117,603ce7100a7217c72c000405,25.0,COMPLETE_NONPARTNER_RECEIPT,2021-03-01 13:07:28,2021-03-01 13:07:28,NaT,2021-03-01 13:07:29,NaT,25.0,2020-08-17 00:00:00,2.0,REJECTED,34.96,5fc961c3b8cfca11a077dd33


In [ ]:
#df_receipt.to_csv('df_receipts.csv', index=False)

### Product (Generated from Receipts)

As mentioned above that there's a nested list under ```rewardsReceiptItemList```column with multiple-to-one relationship to receipts data. To have better understaning of the information contained in that column, we generated a new  ```Product```table with ```receipt_id``` as a foreign key to ```receipts``` table.


In [4]:
product = []
with open('receipts.json') as file:
 for line in file:
  row = json.loads(line)
  
  for p in row.get('rewardsReceiptItemList', [ ]):
    #add '_id' from receipt to product list as foregin key to receipt table
    p['receipt_id'] = row['_id']['$oid']
    product.append(p)


#transform to dataframe
df_product = pd.DataFrame(product)


#reorder the 'receipt_id' column to the front

df_product = df_product[ ['receipt_id'] + [ col for col in df_product.columns if col != 'receipt_id' ] ]
df_product

,receipt_id,barcode,description,finalPrice,itemPrice,needsFetchReview,partnerItemId,preventTargetGapPoints,quantityPurchased,userFlaggedBarcode,...,itemNumber,originalMetaBriteQuantityPurchased,pointsEarned,targetPrice,competitiveProduct,originalFinalPrice,originalMetaBriteItemPrice,deleted,priceAfterCoupon,metabriteCampaignId
0,5ff1e1eb0a720f0523000575,4011,ITEM NOT FOUND,26.00,26.00,False,1,True,5.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5ff1e1bb0a720f052300056b,4011,ITEM NOT FOUND,1,1,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5ff1e1bb0a720f052300056b,028400642255,DORITOS TORTILLA CHIP SPICY SWEET CHILI REDUCE...,10.00,10.00,True,2,True,1.0,028400642255,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5ff1e1f10a720f052300057a,NaN,NaN,NaN,NaN,False,1,True,NaN,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5ff1e1ee0a7214ada100056f,4011,ITEM NOT FOUND,28.00,28.00,False,1,True,4.0,4011,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6936,603cc2bc0a720fde100003e9,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6937,603cc0630a720fde100003e6,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN
6938,603cc0630a720fde100003e6,B07BRRLSVC,thindust summer face mask - sun protection nec...,11.99,11.99,NaN,1,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99,NaN
6939,603ce7100a7217c72c000405,B076FJ92M4,mueller austria hypergrind precision electric ...,22.97,22.97,NaN,0,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.97,NaN


In [ ]:
#df_product.to_csv('df_product.csv', index=False)

### Users

The structure of ```users.json``` is pretty similar with ```receipts.json``` but simpler without nested list, so we can apply the same method we previously used.



In [5]:
users = []
with open('users.json') as file:
 for line in file:
  row = json.loads(line)
  users.append({'_id': row['_id']['$oid'],
   'active': row.get('active', None),
   'createdDate': row.get('createdDate', {}).get('$date', None),
   'lastLogin': row.get('lastLogin', {}).get('$date', None),
   'role': row.get('role', None),
   'signUpSource':row.get('role', None),
   'state':row.get('state', None)})


df_users = pd.DataFrame(users)
df_users

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,consumer,WI
1,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,consumer,WI
2,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,consumer,WI
3,5ff1e1eacfcf6c399c274ae6,True,1609687530554,1.609688e+12,consumer,consumer,WI
4,5ff1e194b6a9d73a3a9f1052,True,1609687444800,1.609688e+12,consumer,consumer,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,fetch-staff,None
491,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,fetch-staff,None
492,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,fetch-staff,None
493,54943462e4b07e684157a532,True,1418998882381,1.614963e+12,fetch-staff,fetch-staff,None


In [6]:
# convert date format

date_col2 = ['createdDate', 'lastLogin']
for i in date_col2:
  df_users[i] = DateFormat(df_users,i)

df_users

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,consumer,WI
1,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,consumer,WI
2,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,consumer,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,consumer,WI
4,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,consumer,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None


In [ ]:
#df_users.to_csv('df_users.csv', index=False)

### Brands
To have better understaning of the information contained in 'cpg' column, I created a new CPG table with brand_id as a foreign key to brands table, storing information of 'cpg_id' and 'cpg_ref'.


In [7]:
brand = []
with open('brands.json') as file:
 for line in file:
  row = json.loads(line)
  brand.append({
   '_id': row['_id']['$oid'],
   'name': row.get('name', None),
   'category': row.get('category', None),
   'barcode': row.get('barcode', None),
   'brandCode': row.get('brandcode', None),
   'topBrand':row.get('topBrand', None),
  })


#transform to dataframe
df_brand = pd.DataFrame(brand)
df_brand

,_id,name,category,barcode,brandCode,topBrand
0,601ac115be37ce2ead437551,test brand @1612366101024,Baking,511111019862,None,False
1,601c5460be37ce2ead43755f,Starbucks,Beverages,511111519928,None,False
2,601ac142be37ce2ead43755d,test brand @1612366146176,Baking,511111819905,None,False
3,601ac142be37ce2ead43755a,test brand @1612366146051,Baking,511111519874,None,False
4,601ac142be37ce2ead43755e,test brand @1612366146827,Candy & Sweets,511111319917,None,False
...,...,...,...,...,...,...
1162,5f77274dbe37ce6b592e90c0,test brand @1601644365844,Baking,511111116752,None,None
1163,5dc1fca91dda2c0ad7da64ae,Dippin Dots® Cereal,Breakfast & Cereal,511111706328,None,None
1164,5f494c6e04db711dd8fe87e7,test brand @1598639215217,Candy & Sweets,511111416173,None,None
1165,5a021611e4b00efe02b02a57,LIPTON TEA Leaves,Grocery,511111400608,None,False


In [ ]:
#df_brand.to_csv('df_brand.csv', index=False)

### CPG

In [8]:
cpg = []
i=0

with open('brands.json') as file:
 for line in file:
  tb={}
  row = json.loads(line)
  tb['brand_id'] = row['_id']['$oid']
  tb['cpg_id'] =row['cpg']['$id']['$oid']
  tb['cpg_ref'] =  row['cpg']['$ref']
  cpg.append(tb)

df_cpg = pd.DataFrame(cpg)
df_cpg

,brand_id,cpg_id,cpg_ref
0,601ac115be37ce2ead437551,601ac114be37ce2ead437550,Cogs
1,601c5460be37ce2ead43755f,5332f5fbe4b03c9a25efd0ba,Cogs
2,601ac142be37ce2ead43755d,601ac142be37ce2ead437559,Cogs
3,601ac142be37ce2ead43755a,601ac142be37ce2ead437559,Cogs
4,601ac142be37ce2ead43755e,5332fa12e4b03c9a25efd1e7,Cogs
...,...,...,...
1162,5f77274dbe37ce6b592e90c0,5f77274dbe37ce6b592e90bf,Cogs
1163,5dc1fca91dda2c0ad7da64ae,53e10d6368abd3c7065097cc,Cogs
1164,5f494c6e04db711dd8fe87e7,5332fa12e4b03c9a25efd1e7,Cogs
1165,5a021611e4b00efe02b02a57,5332f5f6e4b03c9a25efd0b4,Cogs


In [ ]:
#df_cpg.to_csv('df_cpg.csv', index=False)

# Second: Write a query that directly answers a predetermined question from a business stakeholder
In this part, I query pandas DataFrames using SQLite syntax to answer questions 1, 3 and 4.

In [9]:
#Q1. top 5 brands by receipts scanned for most recent month
q1 = """
WITH 
tb1 AS (SELECT date(MAX(strftime('%Y-%m-%d', dateScanned)), '-30 days') AS mon_start FROM df_receipt),
tb2 AS (SELECT MAX(strftime('%Y%m', dateScanned)) AS mon_end FROM df_receipt)

SELECT p.brandCode, COUNT(DISTINCT(p.receipt_id)) ct_receipt
FROM         df_product p
LEFT JOIN    df_receipt r
ON           r._id =p.receipt_id
WHERE strftime('%Y-%m-%d', r.dateScanned) 
BETWEEN (select mon_start from tb1) AND (select mon_end from tb2)
 AND p.brandCode IS NOT NULL
GROUP BY p.brandCode
ORDER BY ct_receipt desc
LIMIT 5
"""


print(ps.sqldf(q1, locals()))

  brandCode  ct_receipt
0     BRAND           3
1   MISSION           2
2      VIVA           1


In [10]:
#Q3. Considering average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
#Assuming status "Finished" indicates status "Accepted"

q3a = """

SELECT AVG(totalSpent) 
FROM df_receipt
WHERE rewardsReceiptStatus = 'FINISHED'

"""

q3b = """

SELECT AVG(totalSpent) 
FROM df_receipt
WHERE rewardsReceiptStatus = 'REJECTED'

"""
print(ps.sqldf(q3a, locals()))
print(ps.sqldf(q3b, locals()))

   AVG(totalSpent)
0        80.854305
   AVG(totalSpent)
0        23.326056


In [ ]:
# Q3 Ans. Average spend from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ is 57.524 higher than 'Rejected'.

In [11]:

#Q4. When considering total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ or ‘Rejected’, which is greater?
#Assuming status "Finished" indicates status "Accepted"

q4a = """

SELECT SUM(purchasedItemCount)
FROM df_receipt
WHERE rewardsReceiptStatus = 'FINISHED'

"""

q4b = """

SELECT SUM(purchasedItemCount)
FROM df_receipt
WHERE rewardsReceiptStatus = 'REJECTED'

"""
print(ps.sqldf(q4a, locals()))
print(ps.sqldf(q4b, locals()))




   SUM(purchasedItemCount)
0                   8184.0
   SUM(purchasedItemCount)
0                    173.0


In [ ]:
#Q4 ans. Total number of items purchased from receipts with 'rewardsReceiptStatus’ of ‘Accepted’ is 8011 higher than 'Rejected'.

# Third: Evaluate Data Quality Issues in the Data Provided


### Data Quality Issue 1
Users data has duplicates values, while user_id should be unique

In [12]:
#examine duplicate rows
df_users_duplicate = df_users[df_users.duplicated()]
print(df_users_duplicate)

                          _id  active         createdDate           lastLogin  \
1    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04 2021-01-03 15:25:37   
2    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04 2021-01-03 15:25:37   
4    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04 2021-01-03 15:25:37   
5    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04 2021-01-03 15:25:37   
8    5ff1e194b6a9d73a3a9f1052    True 2021-01-03 15:24:04 2021-01-03 15:25:37   
..                        ...     ...                 ...                 ...   
490  54943462e4b07e684157a532    True 2014-12-19 14:21:22 2021-03-05 16:52:23   
491  54943462e4b07e684157a532    True 2014-12-19 14:21:22 2021-03-05 16:52:23   
492  54943462e4b07e684157a532    True 2014-12-19 14:21:22 2021-03-05 16:52:23   
493  54943462e4b07e684157a532    True 2014-12-19 14:21:22 2021-03-05 16:52:23   
494  54943462e4b07e684157a532    True 2014-12-19 14:21:22 2021-03-05 16:52:23   

            role signUpSour

In [13]:
#examine number of duplicates of each _id
df_users.groupby('_id', as_index = 'False').count()

,active,createdDate,lastLogin,role,signUpSource,state
_id,,,,,,
54943462e4b07e684157a532,20,20,20,20,20,0
55308179e4b0eabd8f99caa2,1,1,1,1,1,1
5964eb07e4b03efd0c0f267b,1,1,1,1,1,1
59c124bae4b0299e55b0f330,18,18,18,18,18,18
5a43c08fe4b014fd6b6a0612,8,8,8,8,8,0
...,...,...,...,...,...,...
60255883efa60114d20e5d4e,1,1,1,1,1,1
602558adb5459313e1e9b7ce,1,1,1,1,1,1
602558b1efa60114d20e5dc7,1,1,1,1,1,1


In [14]:
#clean up duplicates
df_users_cleaned = df_users.drop_duplicates()
df_users_cleaned

,_id,active,createdDate,lastLogin,role,signUpSource,state
0,5ff1e194b6a9d73a3a9f1052,True,2021-01-03 15:24:04,2021-01-03 15:25:37,consumer,consumer,WI
3,5ff1e1eacfcf6c399c274ae6,True,2021-01-03 15:25:30,2021-01-03 15:25:30,consumer,consumer,WI
6,5ff1e1e8cfcf6c399c274ad9,True,2021-01-03 15:25:28,2021-01-03 15:25:28,consumer,consumer,WI
7,5ff1e1b7cfcf6c399c274a5a,True,2021-01-03 15:24:39,2021-01-03 15:24:39,consumer,consumer,WI
9,5ff1e1f1cfcf6c399c274b0b,True,2021-01-03 15:25:37,2021-01-03 15:25:37,consumer,consumer,WI
...,...,...,...,...,...,...,...
435,5fc961c3b8cfca11a077dd33,True,2020-12-03 22:08:03,2021-02-26 22:39:16,fetch-staff,fetch-staff,NH
455,5fa41775898c7a11a6bcef3e,True,2020-11-05 15:17:09,2021-03-04 16:02:02,fetch-staff,fetch-staff,None
456,5fa32b4d898c7a11a6bcebce,True,2020-11-04 22:29:33,2021-03-04 07:21:58,fetch-staff,fetch-staff,AL
462,5964eb07e4b03efd0c0f267b,True,2017-07-11 15:13:11,2021-03-04 19:07:49,fetch-staff,fetch-staff,IL


In [16]:
df_users_cleaned._id.value_counts()

5ff1e194b6a9d73a3a9f1052    1
600eea525edb7811cde6c81c    1
600fb1ac73c60b12049027bb    1
600fb1ec73c60b1204902af4    1
600fb1e4a4b74c120bd18777    1
                           ..
6000d4abe2571211db395b5c    1
600056d6bd4dff11dda90a3f    1
60005704bd4dff11dda90a9d    1
6002475cfb296c121a81b98d    1
54943462e4b07e684157a532    1
Name: _id, Length: 212, dtype: int64

### Data Quality Issue 2

The "role" column in Users data was set to be "constant value set to 'consumer' ", while there are several user values with role = 'fetch-staff' in the dataset. I wonder if these with role = 'fetch-staff' should be cleaned up if they are testing data.

In [17]:
df_users['role']

0         consumer
1         consumer
2         consumer
3         consumer
4         consumer
          ...     
490    fetch-staff
491    fetch-staff
492    fetch-staff
493    fetch-staff
494    fetch-staff
Name: role, Length: 495, dtype: object

In [18]:
df_users.loc[df_users['role'] == 'fetch-staff']

,_id,active,createdDate,lastLogin,role,signUpSource,state
395,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54,2021-02-08 16:42:58,fetch-staff,fetch-staff,WI
396,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54,2021-02-08 16:42:58,fetch-staff,fetch-staff,WI
397,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54,2021-02-08 16:42:58,fetch-staff,fetch-staff,WI
398,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54,2021-02-08 16:42:58,fetch-staff,fetch-staff,WI
399,59c124bae4b0299e55b0f330,True,2017-09-19 14:07:54,2021-02-08 16:42:58,fetch-staff,fetch-staff,WI
...,...,...,...,...,...,...,...
490,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None
491,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None
492,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None
493,54943462e4b07e684157a532,True,2014-12-19 14:21:22,2021-03-05 16:52:23,fetch-staff,fetch-staff,None


### Data Quality Issue 3

receipts, product, users and brands table all suffering from missing value issues, especially product table. Furthermore, 'Brandcode' column of brand table is completely blank.

In [20]:
#Checking missing values percentage of each dataframe
def null_percentage(df):
  df_null_percentage = round(df.isnull().sum() * 100 / len(df),2)
  df_null_per_rank =  df_null_percentage.sort_values(ascending = False)
  return df_null_per_rank

#null_percentage(df_receipt)

for i in [df_receipt, df_product, df_brand, df_cpg, df_users]:
  print(null_percentage(i))
  print('---------------------')

pointsAwardedDate          52.01
bonusPointsEarned          51.39
bonusPointsEarnedReason    51.39
finishedDate               49.24
pointsEarned               45.58
purchasedItemCount         43.25
purchaseDate               40.04
totalSpent                 38.87
_id                         0.00
createDate                  0.00
dateScanned                 0.00
modifyDate                  0.00
rewardsReceiptStatus        0.00
userId                      0.00
dtype: float64
---------------------
deleted                               99.87
originalMetaBriteItemPrice            99.87
originalFinalPrice                    99.87
originalMetaBriteDescription          99.86
originalMetaBriteQuantityPurchased    99.78
originalMetaBriteBarcode              98.98
itemNumber                            97.80
userFlaggedDescription                97.05
needsFetchReviewReason                96.84
competitorRewardsGroup                96.04
userFlaggedPrice                      95.69
userFlaggedQuanti

#Fourth: Communicate with Stakeholders



Hi all,

Thanks for providing me with the receipts, users and brands datasets. I've been exploring these data and would like to discuss and seek clarifications on the issues I found. 

1. I noticed there are a lot of missing values in each data, especially in the "rewardsReceiptItemList" of receipts data. For example, more than half of the items don't have correlated "barcode", while barcode is critical for building the relationship between receipts and brands data to contruct data model. Does missing value means anything that I'm not aware of?  I wonder if there's a more complete dataset, or if you can tell me about how the data is being collected, we can work on this together and see if the problem is caused during data collecting process or by user side.

2. "BrandCode" of brands data is completely empty. Without information of this column, I wasn't able to tell whether its definition is the same with "BrandCode" in "rewardsReceiptItemList". It would be great if you could clarify on the relationship.


3. The users data contained a lot of duplicates, and I cleaned up those duplicated rows with assumption that each user data should be unique. Please let me know if it's not the case.

Please let me know if you have any questions and I’d be happy to set up a call!


Best,

Tania Kuo